In [ ]:
import json
import os
import uuid

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

In [ ]:
!  mkdir -p data && cd data && wget -c --quiet https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz && tar -xvzf cifar-10-python.tar.gz

In [ ]:
! mkdir -p saved_models

In [ ]:
! pip install fairscale

## Run locally multiGPU training

In [5]:
! cd /home/ec2-user/SageMaker/autoscaler-external/autoscaler/pytorch && python setup.py install
#  Install conda as some fp16 hooks are not supported 
! conda install pytorch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 cudatoolkit=10.2 -c pytorch

running install
running bdist_egg
running egg_info
writing src/autoscaler.egg-info/PKG-INFO
writing dependency_links to src/autoscaler.egg-info/dependency_links.txt
writing top-level names to src/autoscaler.egg-info/top_level.txt
reading manifest file 'src/autoscaler.egg-info/SOURCES.txt'
writing manifest file 'src/autoscaler.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/automl
creating build/bdist.linux-x86_64/egg/automl/optim
copying build/lib/automl/optim/__init__.py -> build/bdist.linux-x86_64/egg/automl/optim
copying build/lib/automl/optim/adamw.py -> build/bdist.linux-x86_64/egg/automl/optim
copying build/lib/automl/optim/optimizer.py -> build/bdist.linux-x86_64/egg/automl/optim
copying build/lib/automl/optim/_functional.py -> build/bdist.linux-x86_64/egg/automl/optim
copying build/lib/automl/__init__.py -> build/bdist.linux-x86_64/egg/a

In [17]:
! pip install tensorboard

     |████████████████████████████████| 5.8 MB 6.4 MB/s            
     |████████████████████████████████| 781 kB 61.7 MB/s            
     |████████████████████████████████| 97 kB 12.1 MB/s            
     |████████████████████████████████| 126 kB 89.1 MB/s            
     |████████████████████████████████| 4.1 MB 60.2 MB/s            
     |████████████████████████████████| 4.9 MB 51.9 MB/s            
     |████████████████████████████████| 156 kB 79.9 MB/s            
     |████████████████████████████████| 155 kB 84.6 MB/s            
     |████████████████████████████████| 151 kB 83.5 MB/s            
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.7.0
    Uninstalling importlib-metadata-3.7.0:
      Successfully uninstalled importlib-metadata-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker

In [ ]:
! python -m torch.distributed.launch --nproc_per_node=8 --nnodes=1 --node_rank=0 \
        /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm/trainer_adascale.py  --num_epochs 200 \
        --batch_size 256 \
        --use_adascale \

        --autoscaler_cfg autoscaler.yaml

The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm/trainer_adascale.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 8
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent

In [ ]:
# Run Sagemaker ddp training

In [ ]:
! pwd

In [ ]:
! ls /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm

In [23]:
! cd /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm &&  python ddp-launcher.py --gpus 8 \
--data_dir /home/ec2-user/SageMaker/data/ \
--model_dir /home/ec2-user/SageMaker/ \
--num_epochs 10


### Data directory:  ['cifar-10-batches-py', 'cifar-10-python.tar.gz']
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : cifar.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 8
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:7777
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agen

## Multinode multi GPU

In [ ]:
config = {
    'batch_size': 256,
    'num_epochs' : 50}

In [ ]:
bucket = 'mansmane-us-west-2'


In [ ]:
# Training time of this job 
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

job = PyTorch(
    entry_point='ddp-launcher.py',
    source_dir='/home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm',
    role=get_execution_role(),
    framework_version='1.8.1',
    instance_count=1,
    instance_type='ml.p3.16xlarge',
    base_job_name='resnet-multi-GPU-g5',
    py_version='py36',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),  # S3 destination of /opt/ml/checkpoints files
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token), # source_dir code will be staged in S3 there
    environment={"SMDEBUG_LOG_LEVEL":"off"},  # reduce verbosity of Debugger
    debugger_hook_config=False,  # deactivate debugger to avoid warnings in model artifact
    disable_profiler=True,  # keep running resources to a minimum to avoid permission errors
    metric_definitions=[
        {"Name": "Train_loss", "Regex": "Training_loss: ([0-9.]+).*$"},
        {"Name": "Learning_rate", "Regex": "learning rate: ([0-9.]+).*$"},        
        {"Name": "Val_loss", "Regex": "Val_loss: ([0-9.]+).*$"},        
        {"Name": "Throughput", "Regex": "Throughput: ([0-9.]+).*$"},
        {"Name": "Val_pixel_acc", "Regex": "Val_pixel_acc: ([0-9.]+).*$"}
    ],
    tags=[{'Key': 'Project', 'Value': 'A2D2_segmentation'}])  # tag the job for experiment tracking

In [ ]:
train_path = 's3://mansmane-us-west-2/cifar10/'

In [ ]:
job.fit({'dataset': train_path}, wait=False)

In [ ]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3
instance_count = 2

job = PyTorch(
    entry_point='ddp-launcher.py',
    source_dir='/home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm',
    role=get_execution_role(),
    framework_version='1.8.1',
    instance_count=instance_count,
    instance_type='ml.p3.16xlarge',
    base_job_name='resnet-multi-GPU-g5-instance-' + str(instance_count),
    py_version='py36',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),  # S3 destination of /opt/ml/checkpoints files
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token), # source_dir code will be staged in S3 there
    environment={"SMDEBUG_LOG_LEVEL":"off"},  # reduce verbosity of Debugger
    debugger_hook_config=False,  # deactivate debugger to avoid warnings in model artifact
    disable_profiler=True,  # keep running resources to a minimum to avoid permission errors
    metric_definitions=[
        {"Name": "Train_loss", "Regex": "Training_loss: ([0-9.]+).*$"},
        {"Name": "Learning_rate", "Regex": "learning rate: ([0-9.]+).*$"},        
        {"Name": "Val_loss", "Regex": "Val_loss: ([0-9.]+).*$"},        
        {"Name": "Throughput", "Regex": "Throughput: ([0-9.]+).*$"},
        {"Name": "Val_pixel_acc", "Regex": "Val_pixel_acc: ([0-9.]+).*$"}
    ],
    tags=[{'Key': 'Project', 'Value': 'A2D2_segmentation'}])  # tag the job for experiment tracking

train_path = 's3://mansmane-us-west-2/cifar10/'
job.fit({'dataset': train_path}, wait=False)